Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



### Import

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold

np.random.seed(42)

### Базовые функции для скоринга и получения рекомендаций

In [2]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [3]:
df = pd.read_csv('train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [4]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [5]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [6]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [50]:
# определим набор данных для GridSearchCV
sim_options = {
    "name": ["msd", "cosine"], # способы оценки похожести (в GridSearch)
    "min_support": [3, 4, 5, 6],     # минимальное кол-во общих пользоватлей с данной шуткой
    "user_based": [False],     # поиск "похожести" будет на основе шуток, а не пользователей
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -1)
gs.fit(data)
     
# результат
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

4.2165999167810995
{'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


In [51]:
# обучим с лучшими параметрами
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 3.1630


3.1630155752190734

In [7]:
from surprise import SVD
from surprise.model_selection import cross_validate

Проверим алгоритм по кросс-валидации "из коробки".

In [10]:
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], n_jobs = -1, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.2140  4.2113  4.2283  4.2157  4.2228  4.2184  0.0062  
MAE (testset)     3.2687  3.2632  3.2783  3.2641  3.2693  3.2687  0.0054  
Fit time          23.52   23.72   23.67   23.65   23.53   23.62   0.08    
Test time         1.94    2.05    1.98    1.85    1.77    1.92    0.10    


{'test_rmse': array([4.21395092, 4.21134903, 4.22830917, 4.21567565, 4.22279362]),
 'test_mae': array([3.26869739, 3.26320166, 3.27825164, 3.26407215, 3.26929574]),
 'fit_time': (23.516319513320923,
  23.724047899246216,
  23.66774845123291,
  23.645395755767822,
  23.525880575180054),
 'test_time': (1.9411273002624512,
  2.054489850997925,
  1.980290174484253,
  1.8510627746582031,
  1.7710673809051514)}

Теперь подберем значения гиперпараметров по сетке.

In [36]:
param_grid = {"n_epochs": [5, 10, 15], "lr_all": [0.002, 0.005, 0.008, 0.01], "reg_all": [0.1, 0.2, 0.3, 0.4, 0.6, 0.8]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -1)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

4.051781460267707
{'n_epochs': 15, 'lr_all': 0.002, 'reg_all': 0.1}


In [37]:
# обучим с лучшими параметрами
svd = gs.best_estimator['rmse']
svd.fit(trainset)

# получим предикт и посмотрим метрику
predictions_svd = svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 3.1204


3.12038806136564

Ошибка на кросс-валидации стала меньше. Уточним значения гиперпараметров.

In [38]:
param_grid = {"n_epochs": [12, 15, 17], "lr_all": [0.001, 0.002], "reg_all": [0.05, 0.07, 0.1]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -1)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

4.0505930677143445
{'n_epochs': 12, 'lr_all': 0.002, 'reg_all': 0.1}


In [39]:
svd = gs.best_estimator['rmse']
svd.fit(trainset)

predictions_svd = svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 3.0841


3.084144782480551

In [16]:
param_grid = {"n_epochs": [11, 12, 13, 14], "lr_all": [0.002, 0.003, 0.004], "reg_all": [0.1, 0.15]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -1)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

4.050286136265535
{'n_epochs': 14, 'lr_all': 0.002, 'reg_all': 0.1}


In [17]:
svd = gs.best_estimator['rmse']
svd.fit(trainset)

predictions_svd = svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 4.0557


4.055660211192525

In [18]:
param_grid = {"n_epochs": [12, 14, 16], "lr_all": [0.002, 0.0025], "reg_all": [0.1, 0.11, 0.12, 0.13, 0.14]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -1)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

4.045473989186674
{'n_epochs': 16, 'lr_all': 0.002, 'reg_all': 0.13}


In [19]:
svd = gs.best_estimator['rmse']
svd.fit(trainset)

predictions_svd = svd.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 4.0509


4.050920928319449

Предсказание рейтинга при использовании алгоритма на основе SVD дало сильно меньшую ошибку RMSE на тестовой выборке и вероятно будет давать более качественные пресказания.

### Тестирование и результаты

In [28]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # id шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = -3.08   {'actual_k': 40, 'was_impossible': False}


In [20]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # id шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = svd.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = -3.75   {'was_impossible': False}


In [29]:
uid = 24983  # id пользователя 
iid = 62     # id шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = 5.19   {'actual_k': 40, 'was_impossible': False}


In [21]:
uid = 24983  # id пользователя 
iid = 62     # id шутки

pred = svd.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = 4.87   {'was_impossible': False}


При выборочной проверке случайных пользователей, мы видим более точные предсказания при использовании алгоритма на основе SVD.

### Обзор рекомендаций

In [22]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
#trainset = algo.trainset

predictions_df = pd.DataFrame(predictions_svd, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [23]:
best_predictions.head(10)

,uid,iid,rui,est,details,№ кол-во пользовательских рейтингов,№ кол-во рейтингов элементов,error
56247,23533,69,2.77,2.769993,{'was_impossible': False},43,15997,0.000007
120998,628,26,1.02,1.019988,{'was_impossible': False},52,15140,0.000012
220412,15140,96,1.46,1.460014,{'was_impossible': False},55,6631,0.000014
144464,9244,46,-0.39,-0.390031,{'was_impossible': False},29,15041,0.000031
202671,11979,32,0.87,0.870037,{'was_impossible': False},30,16029,0.000037
281414,22841,36,2.57,2.569963,{'was_impossible': False},57,15847,0.000037
115623,4230,39,-0.53,-0.529955,{'was_impossible': False},66,14817,0.000045
148095,6327,64,1.21,1.209908,{'was_impossible': False},40,11146,0.000092
105142,10024,57,-5.44,-5.440102,{'was_impossible': False},67,10208,0.000102
181277,561,33,1.89,1.889805,{'was_impossible': False},48,10767,0.000195


In [24]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions_svd = svd.test(testset)

top_n = get_top_n(predictions_svd)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

19208 [54, 65, 89, 29, 81]
8671 [8, 83, 32, 19, 72]
6037 [27, 36, 87, 89, 80]
3233 [50, 31, 36, 27, 42]
3449 [100, 65, 83, 80, 53]
10032 [35, 32, 53, 29, 91]
5774 [89, 91, 29, 32, 88]
23392 [53, 62, 36, 89, 21]
3039 [89, 91, 62, 36, 88]
17395 [27, 89, 11, 10, 50]


### Для отправки на тестирование

Обучим модель на полной тестовой выборке

In [25]:
svd.fit(trainset_data)

In [26]:
test = pd.read_csv('test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [27]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: svd.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [28]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,3.236721
1,-8.130317
2,-1.212230
3,6.749977
4,6.252820


In [29]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline.csv')